In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
from scipy import stats
import scipy.stats

In [17]:
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding='latin1')
df.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [18]:
print(df.columns)
print(df.shape)
print(df.dtypes)
df.describe()

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'],
      dtype='object')
(2127, 5)
PMID/PMCID                                             object
Publisher                                              object
Journal title                                          object
Article title                                          object
COST (£) charged to Wellcome (inc VAT when charged)    object
dtype: object


,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
count,1928,2127,2126,2127,2127
unique,1880,299,984,2126,1402
top,In Process,Elsevier,PLoS One,"Exclusive breastfeeding, diarrhoel morbidity a...",£2040.00
freq,7,387,92,2,94


In [28]:
df.rename(columns={'PMID/PMCID':'PMID', 'Journal title': 'journal_title', 
                    'Article title': 'article_title', 'COST (£) charged to Wellcome (inc VAT when charged)': 'cost'}, inplace=True)

In [29]:
df['journal_title'].unique

<bound method Series.unique of 0                                  Psychological Medicine
1                                       Biomacromolecules
2                                              J Med Chem
3                                              J Med Chem
4                                              J Org Chem
5                          Journal of Medicinal Chemistry
6                            Journal of Proteome Research
7                                               Mol Pharm
8                                    ACS Chemical Biology
9                                    ACS Chemical Biology
10           Journal of Chemical Information and Modeling
11                         Journal of Medicinal Chemistry
12                         Journal of Medicinal Chemistry
13                                           Biochemistry
14                         Journal of Medicinal Chemistry
15                                       Gastroenterology
16                        Journal of Biol

In [30]:
print(df['journal_title'].apply(lambda x: str(x).isalpha()))

0       False
1        True
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13       True
14      False
15       True
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2097    False
2098    False
2099    False
2100    False
2101    False
2102     True
2103    False
2104     True
2105     True
2106    False
2107    False
2108    False
2109     True
2110     True
2111     True
2112     True
2113     True
2114    False
2115     True
2116    False
2117    False
2118    False
2119    False
2120    False
2121    False
2122    False
2123     True
2124    False
2125     True
2126    False
Name: journal_title, Length: 2127, dtype: bool


In [31]:
df['journal_title'].isnull().sum()

1

In [32]:
df['journal_title'].dropna()
# the length changes to 2126 since one title is dropped

0                                  Psychological Medicine
1                                       Biomacromolecules
2                                              J Med Chem
3                                              J Med Chem
4                                              J Org Chem
5                          Journal of Medicinal Chemistry
6                            Journal of Proteome Research
7                                               Mol Pharm
8                                    ACS Chemical Biology
9                                    ACS Chemical Biology
10           Journal of Chemical Information and Modeling
11                         Journal of Medicinal Chemistry
12                         Journal of Medicinal Chemistry
13                                           Biochemistry
14                         Journal of Medicinal Chemistry
15                                       Gastroenterology
16                        Journal of Biological Chemistry
17            

In [38]:
lower_titles = df['journal_title'].apply(lambda x: str(x).lower())
print(lower_titles)

0                                  psychological medicine
1                                       biomacromolecules
2                                              j med chem
3                                              j med chem
4                                              j org chem
5                          journal of medicinal chemistry
6                            journal of proteome research
7                                               mol pharm
8                                    acs chemical biology
9                                    acs chemical biology
10           journal of chemical information and modeling
11                         journal of medicinal chemistry
12                         journal of medicinal chemistry
13                                           biochemistry
14                         journal of medicinal chemistry
15                                       gastroenterology
16                        journal of biological chemistry
17            

In [40]:
lower_titles.nunique

<bound method IndexOpsMixin.nunique of 0                                  psychological medicine
1                                       biomacromolecules
2                                              j med chem
3                                              j med chem
4                                              j org chem
5                          journal of medicinal chemistry
6                            journal of proteome research
7                                               mol pharm
8                                    acs chemical biology
9                                    acs chemical biology
10           journal of chemical information and modeling
11                         journal of medicinal chemistry
12                         journal of medicinal chemistry
13                                           biochemistry
14                         journal of medicinal chemistry
15                                       gastroenterology
16                        journal

In [47]:
df.loc[:, 'Publisher'] = df['Publisher'].str.strip()
df.loc[:, 'journal_title'] = df['journal_title'].str.strip()
df.loc[:, 'article_title'] = df['article_title'].str.strip()
df.loc[:, 'cost'] = df['cost'].str.strip()

In [48]:
df['journal_title'].nunique()

951

In [49]:
df[df.duplicated('article_title', keep=False)]

,PMID,Publisher,journal_title,article_title,cost
1490,Pending,Public Library of Science,PLoS One,"Exclusive breastfeeding, diarrhoel morbidity a...",£825.68
1496,NaN,Public Library of Science,PLoS One,"Exclusive breastfeeding, diarrhoel morbidity a...",£825.68


In [50]:
df = df.drop_duplicates('article_title')

In [53]:
df['journal_title'].nunique()

951

In [54]:
df['journal_title'].unique()

array(['Psychological Medicine', 'Biomacromolecules', 'J Med Chem',
       'J Org Chem', 'Journal of Medicinal Chemistry',
       'Journal of Proteome Research', 'Mol Pharm',
       'ACS Chemical Biology',
       'Journal of Chemical Information and Modeling', 'Biochemistry',
       'Gastroenterology', 'Journal of Biological Chemistry',
       'Journal of Immunology', 'ACS Chemical Neuroscience', 'ACS NANO',
       'American Chemical Society', 'Analytical Chemistry',
       'Bioconjugate Chemistry',
       'Journal of the American Chemical Society', 'ACS Nano', 'CHEST',
       'Journal of Neurophysiology', 'Journal of Physiology',
       'The Journal of Neurophysiology', 'American Journal of Psychiatry',
       'Americal Journal of Psychiatry', 'Behavioral Neuroscience',
       'Emotion', 'Health Psychology', 'Journal of Abnormal Psychology',
       'Journal of Consulting and Clinical Psychology',
       'Journal of Experimental Psychology:  Animal Behaviour Process',
       'Journal o

In [69]:
df['journal_title'].describe()

count         2125
unique         951
top       PLoS One
freq            91
Name: journal_title, dtype: object

In [60]:
df.groupby('journal_title').describe().head(5).reset_index()

journal_title  PMID                                          \
                             count unique                         top freq   
0       ACS Chemical Biology     4      4  PMID: 24015914 PMC3833349     1   
1  ACS Chemical Neuroscience     1      1    PMCID:\n    PMC3656742\n    1   
2                   ACS NANO     1      1              PMCID: 3584654    1   
3                   ACS Nano     1      1                    23373658    1   
4                     ACTA F     1      1                  PMC3374517    1   

  Publisher                                                     article_title  \
      count unique                                     top freq         count   
0         5      2               American Chemical Society    3             5   
1         1      1               American Chemical Society    1             1   
2         1      1               AMERICAN CHEMICAL SOCIETY    1             1   
3         1      1  American Chemical Society Publications    1             1   
4         1      1  International Union of Crystallography    1             1   

                                                                  cost         \
  unique                                                top freq count unique   
0      5  Discovery of ?2 Adrenergic Receptor Ligands Us...    1     5      5   
1      1  Continuous online microdialysis using microflu...    1     1      1   
2      1  HYDROXY-TERMINATED CONJUGATED POLYMER NANOPART...    1     1      1   
3      1  Skin dendritic cell targeting via microneedle ...    1     1      1   
4      1  Crystallization and preliminary crystallograph...    1     1      1   

                  
        top freq  
0  £1294.78    1  
1  £1186.80    1  
2   £642.89    1  
3   £693.39    1  
4   £754.90    1

In [61]:
lower_titles.str.strip()

0                                  psychological medicine
1                                       biomacromolecules
2                                              j med chem
3                                              j med chem
4                                              j org chem
5                          journal of medicinal chemistry
6                            journal of proteome research
7                                               mol pharm
8                                    acs chemical biology
9                                    acs chemical biology
10           journal of chemical information and modeling
11                         journal of medicinal chemistry
12                         journal of medicinal chemistry
13                                           biochemistry
14                         journal of medicinal chemistry
15                                       gastroenterology
16                        journal of biological chemistry
17            

In [65]:
unique_titles = lower_titles.unique()

In [68]:
print(unique_titles)

['psychological medicine' 'biomacromolecules' 'j med chem' 'j org chem'
 'journal of medicinal chemistry' 'journal of proteome research'
 'mol pharm' 'acs chemical biology'
 'journal of chemical information and modeling' 'biochemistry'
 'gastroenterology' 'journal of biological chemistry'
 'journal of immunology' 'acs chemical neuroscience' 'acs nano'
 'american chemical society' 'analytical chemistry'
 'bioconjugate chemistry' 'journal of medicinal chemistry '
 'journal of the american chemical society' 'chest'
 'journal of neurophysiology' 'journal of physiology'
 'the journal of neurophysiology' 'american journal of psychiatry'
 'americal journal of psychiatry' 'behavioral neuroscience' 'emotion'
 'health psychology' 'journal of abnormal psychology'
 'journal of consulting and clinical psychology'
 'journal of experimental psychology:  animal behaviour process'
 'journal of experimental psychology: human perception and performance'
 'journal of family psychology' 'psychological asse

In [76]:
lower_titles.str.strip().value_counts(dropna=False)[:5]

#5 most popular journals and their article counts

plos one                           190
journal of biological chemistry     53
neuroimage                          29
nucleic acids research              26
plos genetics                       24
Name: journal_title, dtype: int64

In [77]:
df['cost'] = df['cost'].map(lambda x: x.lstrip('£').rstrip('£'))

In [78]:
df['cost'] = df['cost'].str.extract('(\d+)', expand=False)

In [114]:
df['cost'] = df['cost'].astype(int)
scipy.stats.mstats.winsorize(df['cost'], limits = 0.05)

#changes costs to an integer and then removes outliers (winsorize)

masked_array(data=[ 734, 2381,  734, ..., 1834, 2374, 2034],
             mask=False,
       fill_value=999999)

In [120]:
#df['cost'].fillna()

df.groupby('cost').std().fillna(df.groupby('cost').last())

##how can I drop null values for standard deviation

DataError: No numeric types to aggregate

In [109]:
df.groupby(['journal_title', 'article_title'])['cost'].agg(['mean', 'median', 'std'])

mean  \
journal_title                                      article_title                                                
ACS Chemical Biology                               A Novel Allosteric Inhibitor of the Uridine Dip...    1294   
                                                   Chemical proteomic analysis reveals the drugabi...    1294   
                                                   Discovery of ?2 Adrenergic Receptor Ligands Usi...     947   
                                                   Discovery of an allosteric inhibitor binding si...    1267   
                                                   Synthesis of alpha-glucan in mycobacteria invol...    2286   
ACS Chemical Neuroscience                          Continuous online microdialysis using microflui...    1186   
ACS NANO                                           HYDROXY-TERMINATED CONJUGATED POLYMER NANOPARTI...     642   
ACS Nano                                           Skin dendritic cell targeting via microneedle a...     693   
ACTA F                                             Crystallization and preliminary crystallographi...     754   
AGE                                                Voluntary exercise can strengthen the circadian...    2002   
AIDS                                               Evaluation of an empiric risk screening score t...    1834   
                                                   Grassroots Community Organisations' Contributio...    2374   
                                                   HIV infected adolescents in Southern Africa can...    1968   
AIDS Behav                                         Adherence to antiretroviral therapy and clinica...    1834   
AIDS Care                                          Burden of HIV among primary school children and...    2145   
                                                   WORLDBANK Special Issue: Evidence for a contrib...    2232   
AIDS Journal                                       Short title: TB and VL breakthrough and failure...    2015   
AIDS Research and Therapy                          Collective Patient behaviours derailing ART roo...    1240   
AIDS UK                                            HIV incidence and survival from age-specific se...    1836   
                                                   Sexual behaviour in a rural high HIV prevalence       1836   
ASN Neuro                                          Astrocytes and glutamate homeostasis in Alzheim...    1453   
Academy of Nutrition and Dietetics                 Parent support and parent mediated behaviours a...    2379   
Acta Crystallographica Section D,  Biological C... Clustering procedures for the optimal selection...     771   
Acta Crystallographica Section D: Biological Cr... Intensity statistics in the presence of transla...     773   
Acta Crystallographica Section F: Structural Bi... Structure of Pseudomonas aeruginosa inosine 5'-...     807   
                                                   Structure of diaminohydroxyphosphoribosylaminop...     785   
Acta Crystallographica, Section D                  Nearest-cell: a fast and easy tool for locating...     757   
Acta Crystallography D                             Crystallization, dehydration and experimental p...     774   
Acta D                                             Structure of HLA-A*0301 in complex with a pepti...     750   
Acta Dermato Venereologica                         The Importance of a Full Clinical Examination: ...     653   
...                                                                                                       ...   
Tropical Medicine and International Health         Epidemiology and control of trachoma: systamati...    1974   
                                                   Maternal recall of birth weight and birth size ...    1896   
                                                   Meningococcal carriage in the African meningiti...    1949   
                                     

In [111]:
df.groupby(['journal_title', 'article_title'])['cost'].describe()

count  \
journal_title                                      article_title                                               
ACS Chemical Biology                               A Novel Allosteric Inhibitor of the Uridine Dip...    1.0   
                                                   Chemical proteomic analysis reveals the drugabi...    1.0   
                                                   Discovery of ?2 Adrenergic Receptor Ligands Usi...    1.0   
                                                   Discovery of an allosteric inhibitor binding si...    1.0   
                                                   Synthesis of alpha-glucan in mycobacteria invol...    1.0   
ACS Chemical Neuroscience                          Continuous online microdialysis using microflui...    1.0   
ACS NANO                                           HYDROXY-TERMINATED CONJUGATED POLYMER NANOPARTI...    1.0   
ACS Nano                                           Skin dendritic cell targeting via microneedle a...    1.0   
ACTA F                                             Crystallization and preliminary crystallographi...    1.0   
AGE                                                Voluntary exercise can strengthen the circadian...    1.0   
AIDS                                               Evaluation of an empiric risk screening score t...    1.0   
                                                   Grassroots Community Organisations' Contributio...    1.0   
                                                   HIV infected adolescents in Southern Africa can...    1.0   
AIDS Behav                                         Adherence to antiretroviral therapy and clinica...    1.0   
AIDS Care                                          Burden of HIV among primary school children and...    1.0   
                                                   WORLDBANK Special Issue: Evidence for a contrib...    1.0   
AIDS Journal                                       Short title: TB and VL breakthrough and failure...    1.0   
AIDS Research and Therapy                          Collective Patient behaviours derailing ART roo...    1.0   
AIDS UK                                            HIV incidence and survival from age-specific se...    1.0   
                                                   Sexual behaviour in a rural high HIV prevalence       1.0   
ASN Neuro                                          Astrocytes and glutamate homeostasis in Alzheim...    1.0   
Academy of Nutrition and Dietetics                 Parent support and parent mediated behaviours a...    1.0   
Acta Crystallographica Section D,  Biological C... Clustering procedures for the optimal selection...    1.0   
Acta Crystallographica Section D: Biological Cr... Intensity statistics in the presence of transla...    1.0   
Acta Crystallographica Section F: Structural Bi... Structure of Pseudomonas aeruginosa inosine 5'-...    1.0   
                                                   Structure of diaminohydroxyphosphoribosylaminop...    1.0   
Acta Crystallographica, Section D                  Nearest-cell: a fast and easy tool for locating...    1.0   
Acta Crystallography D                             Crystallization, dehydration and experimental p...    1.0   
Acta D                                             Structure of HLA-A*0301 in complex with a pepti...    1.0   
Acta Dermato Venereologica                         The Importance of a Full Clinical Examination: ...    1.0   
...                                                                                                      ...   
Tropical Medicine and International Health         Epidemiology and control of trachoma: systamati...    1.0   
                                                   Maternal recall of birth weight and birth size ...    1.0   
                                                   Meningococcal carriage in the African meningiti...    1.0   
                                                   Nutritional suppleme